## Adaptive Pooling

Adaptive average/max pooling is great when you have a pre-trained backbone and want to build a custom head. Because the output of the backbone layer could change you want to resize it to specific dimension so you can handle any input to your head.

In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [92]:
alexnet = models.mobilenet_v2(pretrained=True)

Here we remove the classification layer of the pretrained AlexNet. We only use AlexNet as a feature extractor. So we will remove the fully connected layer at the top of the model. Now the final layer of AlexNet is the last conv layer.

In [93]:
backbone = nn.Sequential(list(alexnet.children())[0])

Print out the last layers:

In [111]:
backbone[-1][-1]

ConvBNReLU(
  (0): Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace=True)
)

In [96]:
x = torch.randn(3,3,280,280)

In [101]:
y = backbone(x)
y.shape

torch.Size([3, 1280, 9, 9])

## Head

The first layer of our head will be the nn.AdaptiveAvgPool3d. Here we can define the output we want. In this case we want the following output:

**(C, 10, 10, 10)**

Remember that a pooling layer has no parameters. So we don't have to worry about initializing weights or anything.

In [106]:
class Head(nn.Module):
    def __init__(self):
        super(Head, self).__init__()
        self.aap = nn.AdaptiveAvgPool3d((10,10,10))
    
    def forward(self, x):
        return self.aap(x)

In [107]:
h = Head()

In [108]:
h(y).shape

torch.Size([3, 10, 10, 10])